In [1]:
from sqlanalyzer import column_parser, unbundle
import re
import json

In [13]:
query = """WITH reg_users AS\n  (SELECT *\n   FROM\n     (SELECT a.*,\n             b.*,\n             c.*,\n             d.*\n      FROM\n        (SELECT DISTINCT anonymous_id,\n                         user_id\n         FROM mapbox_customer_data.segment_identifies\n         WHERE dt >= '2018-07-01'\n           AND anonymous_id IS NOT NULL\n           AND user_id IS NOT NULL ) a\n      LEFT JOIN\n        (SELECT id,\n                email,\n                created\n         FROM mapbox_customer_data.accounts\n         WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id\n      LEFT JOIN\n        (SELECT anonymous_id AS anon_id_ad,\n                context_campaign_name,\n                min(TIMESTAMP) AS min_exposure\n         FROM mapbox_customer_data.segment_pages\n         WHERE dt >= '2018-07-01'\n           AND context_campaign_name IS NOT NULL\n         GROUP BY 1,\n                  2) c ON a.anonymous_id = c.anon_id_ad\n      LEFT JOIN\n        (SELECT DISTINCT anonymous_id AS anon_id_event,\n                         original_timestamp,\n                         event,\n                         context_traits_email\n         FROM mapbox_customer_data.segment_tracks\n         WHERE dt >= '2018-07-01'\n           AND event LIKE 'submitted_%form'\n           AND context_traits_email IS NOT NULL ) d ON a.anonymous_id = d.anon_id_event)\n   WHERE context_campaign_name IS NOT NULL ),\n\n     non_reg_users AS\n  (SELECT context_campaign_name,\n          min_exposure,\n          event,\n          original_timestamp AS event_timestamp,\n          context_traits_email AS event_email\n   FROM\n     (SELECT a.*,\n             b.*\n      FROM\n        (SELECT anonymous_id AS anon_id_ad,\n                context_campaign_name,\n                min(original_timestamp) AS min_exposure\n         FROM\n           (SELECT context_campaign_name,\n                   anonymous_id,\n                   original_timestamp\n            FROM mapbox_customer_data.segment_pages\n            WHERE dt >= '2018-07-01'\n              AND context_campaign_name IS NOT NULL )\n         GROUP BY 1,\n                  2) a\n      LEFT JOIN\n        (SELECT DISTINCT anonymous_id AS anon_id_event,\n                         original_timestamp,\n                         event,\n                         context_traits_email\n         FROM mapbox_customer_data.segment_tracks\n         WHERE dt >= '2018-07-01'\n           AND event LIKE 'submitted_%form'\n           AND context_traits_email IS NOT NULL ) b ON a.anon_id_ad = b.anon_id_event)\n   WHERE anon_id_event IS NOT NULL\n     AND to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n     AND cast(min_exposure AS DATE) >= cast(original_timestamp AS DATE) - INTERVAL '28' DAY ),\n\n     mql_flag AS\n  (SELECT email,\n          created_date,\n          last_mql_date_c,\n          mql_flag\n   FROM\n     (SELECT email,\n             min(created_date) created_date,\n             max(last_mql_date_c) last_mql_date_c,\n             CASE\n                 WHEN max(last_mql_date_c) IS NOT NULL THEN 1\n                 ELSE 0\n             END AS mql_flag,\n             sum(CASE\n                     WHEN is_deleted = TRUE THEN 1\n                     ELSE 0\n                 END) AS is_deleted\n      FROM sales.salesforce_leads\n      WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY\n      GROUP BY 1)\n   WHERE mql_flag = 1\n     AND is_deleted = 0 ),\n\n     cleaned_list AS\n  (SELECT DISTINCT *\n   FROM\n     (SELECT context_campaign_name,\n             min_exposure,\n             'created_an_account' AS event,\n             created AS event_timestamp,\n             email AS event_email\n      FROM reg_users\n      WHERE to_unixtime(min_exposure) <= to_unixtime(created)\n        AND cast(min_exposure AS DATE) >= cast(created AS DATE) - INTERVAL '28' DAY\n      UNION ALL SELECT context_campaign_name,\n                       min_exposure,\n                       event,\n                       original_timestamp AS event_timestamp,\n                       context_traits_email AS event_email\n      FROM reg_users\n      WHERE to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n        AND cast(min_exposure AS DATE) >= cast(original_timestamp AS DATE) - INTERVAL '28' DAY\n      UNION ALL SELECT *\n      FROM non_reg_users))\n\nSELECT a.*,\n       b.*\nFROM cleaned_list a\nLEFT JOIN mql_flag b ON a.event_email = b.email\n\n"
"""

In [14]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')


In [15]:
cte_dict = formatter.parse_cte(formatted_query)
cte_dict

{'reg_users': "SELECT *\n   FROM\n     (SELECT a.*,\n             b.*,\n             c.*,\n             d.*\n      FROM\n        (SELECT DISTINCT anonymous_id,\n                         user_id\n         FROM mapbox_customer_data.segment_identifies\n         WHERE dt >= '2018-07-01'\n           AND anonymous_id IS NOT NULL\n           AND user_id IS NOT NULL ) a\n      LEFT JOIN\n        (SELECT id,\n                email,\n                created\n         FROM mapbox_customer_data.accounts\n         WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id\n      LEFT JOIN\n        (SELECT anonymous_id AS anon_id_ad,\n                context_campaign_name,\n                min(TIMESTAMP) AS min_exposure\n         FROM mapbox_customer_data.segment_pages\n         WHERE dt >= '2018-07-01'\n           AND context_campaign_name IS NOT NULL\n         GROUP BY 1,\n                  2) c ON a.anonymous_id = c.anon_id_ad\n      LEFT JOIN\n        (SELECT DISTINCT anony